# Brute Force Pipeline for Wildfire Detection - by Selman Tabet @ https://selman.io/

In [1]:
import os
import time


### Environment Setup

In [2]:
print(os.getcwd())

d:\Dissertation\Kaggle Data\wildfire-detection


In [3]:
import socket
print(socket.gethostname())
import os
try: # for CUDA enviroment
    os.system("nvidia-smi")
except:
    pass

Chaos


### Importing Libraries

In [4]:
# Data processing libraries
import numpy as np
from itertools import combinations
import json
import argparse

# ML libraries
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model

# Chart generating libraries
from tensorflow.keras.utils import plot_model

from IPython import get_ipython
from pprint import pprint

# Custom helper libraries
from default_params import *
from wildfirenet import *
from utils.img_processing import resize_and_compress_images
from utils.dataset_processors import *
from utils.plot_functions import *
from utils.evaluator import *

d:\Dissertation\Kaggle Data\wildfire-detection\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Device check

In [5]:
cuda_visible_devices = os.environ.get('CUDA_VISIBLE_DEVICES')
print(f"CUDA_VISIBLE_DEVICES: {cuda_visible_devices}")
print(tf.config.get_visible_devices())
print(tf.config.list_physical_devices('GPU'))

CUDA_VISIBLE_DEVICES: None
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[]


### Parse arguments from command line

In [6]:
# Detect if running in a Jupyter notebook
def in_notebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        else:
            return False  # Other type (terminal, etc.)
    except NameError:
        return False      # Probably standard Python interpreter
    
from_py = False
parser = argparse.ArgumentParser(
    description="Parse command line arguments")
parser.add_argument('--from-py-cfg', type=str,
                    help='Path to the config Python file')
if in_notebook():
    args = parser.parse_args([])  # Ignore sys.argv
else:
    args = parser.parse_args()
config_file_path = args.from_py_cfg
print(f"Python Config Path: {config_file_path}")

if config_file_path:
    import importlib.util
    spec = importlib.util.spec_from_file_location("config_module", config_file_path)
    config_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(config_module)
    config = config_module.cfg
    print("Loaded config from Python file:")
    pprint(config)
    # Datasets, models, and hyperparameters are mandatory and must be processed now.
    training_datasets = config.get('train', {})
    if training_datasets is None or len(training_datasets) == 0:
        raise ValueError("No train datasets defined in config")
    full_test_dir = config.get('test')
    base_models = config.get('keras_models', [])
    custom_models = config.get('custom_models', [])
    if base_models is None or len(base_models) == 0:
        if custom_models is None or len(custom_models) == 0:
            raise ValueError("No models defined in config")
    
    hyperparameters = config.get('hyperparameters')
    default_hyperparameters = default_cfg.get('hyperparameters', {})
    if hyperparameters is None or len(hyperparameters) == 0:
        print("No training hyperparameters defined in config, using defaults")
        hyperparameters = default_hyperparameters
    else:
        for key, value in default_hyperparameters.items():
            if key not in hyperparameters:
                print(f"Missing parameter - falling back to default hyperparameter {key}:{default_hyperparameters[key]}")
                hyperparameters[key] = default_hyperparameters[key]
    from_py = True # Successfully completed the import
else:
    print("No Python config file specified, using default config")
    config = default_cfg
    training_datasets = config.get('train', {})
    base_models = config.get('keras_models', [])
    custom_models = config.get('custom_models', [])
    hyperparameters = config.get('hyperparameters')
    full_test_dir = config.get('test')

Python Config Path: None
No Python config file specified, using default config


### Parsing parameters

In [7]:

train_dirs = [training_datasets[ds].get('train') for ds in training_datasets]
test_dirs = [training_datasets[ds].get('test') for ds in training_datasets]
val_dirs = [training_datasets[ds].get('val') for ds in training_datasets]

# Combine base_models and custom_models
all_models = base_models + custom_models
# Create a list to keep track of which models are custom
is_custom_model = [False] * len(base_models) + [True] * len(custom_models)

print("All models:", all_models)
print("Is custom model:", is_custom_model)

All models: [<function MobileNetV3Small at 0x00000290B3857BA0>, <Sequential name=WildfireNet, built=True>]
Is custom model: [False, True]


### Setting parameters

In [8]:
if from_py:
    epochs = hyperparameters.get('epochs')
    batch_size = hyperparameters.get('batch_size')
    img_height = config.get('image_height', default_cfg.get('image_height'))
    img_width = config.get('image_width', default_cfg.get('image_width'))
    optimizer_fn = config.get('optimizer', default_cfg.get('optimizer'))
    loss_fn = config.get('loss', default_cfg.get('loss'))
    callbacks_list = config.get('callbacks', default_cfg.get('callbacks'))
    metrics_list = config.get('metrics', default_cfg.get('metrics'))
    enforce_image_size = config.get('enforce_resolution', default_cfg.get('enforce_resolution'))
else:
    epochs = hyperparameters.get('epochs')
    batch_size = hyperparameters.get('batch_size')
    img_height = default_cfg.get('image_height')
    img_width = default_cfg.get('image_width')
    optimizer_fn = default_cfg.get('optimizer')
    loss_fn = default_cfg.get('loss')
    callbacks_list = default_cfg.get('callbacks')
    metrics_list = default_cfg.get('metrics')
    enforce_image_size = default_cfg.get('enforce_resolution')

checkpoint_path = os.path.join("checkpoint")


### Enforce defined resolution

In [9]:
if enforce_image_size:
    all_dirs = train_dirs + test_dirs + val_dirs + [full_test_dir]
    all_dirs = [d for d in all_dirs if d is not None]
    for directory in all_dirs:
        print(f"Adjusting resolutions of images in {directory}")
        resize_and_compress_images(directory, target_size=(img_width, img_height))

### Generate training and validation datasets

In [10]:
dataset_names = []
train_datasets = [] # [ (dataset_1_train, dataset_2_train), ... ]
train_sizes = [] # [ (dataset_1_train_size, dataset_2_train_size), ... ]
val_datasets = [] # [ (dataset_1_val, dataset_2_val), ... ]
val_sizes = [] # [ (dataset_1_val_size, dataset_2_val_size), ... ]


for d in training_datasets:
    print(f"Processing: {d}")
    train_dir = training_datasets[d].get('train')
    augment = training_datasets[d].get('augment', True)
    print("Augmenting" if augment else "Not augmenting", d)
    # Apply original and augmented data generators for training
    print("Creating generators for training")
    train_generator, augmented_train_generator = create_generators(train_dir, augment=augment, img_width=img_width, img_height=img_height)
    train_samples = samples_from_generators([train_generator, augmented_train_generator])  
    train_dataset = generators_to_dataset([train_generator, augmented_train_generator], batch_size=batch_size, img_height=img_height, img_width=img_width)
    
    # Apply original and augmented data generators for validation
    if "val" in training_datasets[d]:
        val_dir = training_datasets[d]['val']
        print("Creating generators for validation")
        val_generator, augmented_val_generator = create_generators(val_dir, augment=augment, shuffle=False, img_width=img_width, img_height=img_height)
        val_samples = samples_from_generators([val_generator, augmented_val_generator])
        val_dataset = generators_to_dataset([train_generator, augmented_train_generator], batch_size=batch_size, img_height=img_height, img_width=img_width)
    else:
        print("No validation set, splitting training set.")
        train_dataset, val_dataset, train_samples, val_samples = val_split(train_dataset, train_samples)
        val_generator, augmented_val_generator = None, None
    
    # Calculate the number of samples for training and validation
    train_sizes.append(train_samples)
    val_sizes.append(val_samples)

    show_counts_from_generators(train_generator, augmented_train_generator)

    train_datasets.append(train_dataset)
    val_datasets.append(val_dataset)
    dataset_names.append(d)
    
# Ensure that the lengths are consistent across the board before continuing
assert(len(train_sizes) == len(train_datasets) and len(train_sizes) == len(val_sizes) and len(val_sizes) == len(val_datasets))


Processing: The Wildfire Dataset
Augmenting The Wildfire Dataset
Creating generators for training
Found 1887 images belonging to 2 classes.
Found 1887 images belonging to 2 classes.
Creating generators for validation
Found 402 images belonging to 2 classes.
Found 402 images belonging to 2 classes.
Number of samples in generator: 1887
Number of classes: 2
--------------------
Class indices: {'fire': 0, 'nofire': 1}
Class names: ['fire', 'nofire']
Dataset Class Counts:
fire: 730
nofire: 1157

Augmented Dataset Class Counts:
fire: 730
nofire: 1157


Combined Dataset Class Counts:
fire: 1460
nofire: 2314
--------------------
Processing: DeepFire
Augmenting DeepFire
Creating generators for training
Found 1520 images belonging to 2 classes.
Found 1520 images belonging to 2 classes.
No validation set, splitting training set.
Splitted dataset:
Training dataset size: 2432 samples
Validation dataset size: 608 samples
Number of samples in generator: 1520
Number of classes: 2
--------------------


### Brute Force Combinatorial Search

In [11]:
dataset_combos = [] # [(0,), (1,), (0, 1), ...] where 0, 1 are the indices of the datasets within their respective lists
for r in range(1, len(dataset_names) + 1):
    dataset_combos.extend(combinations(range(len(dataset_names)), r))
combined_training_datasets = []
combined_val_datasets = []
combined_dataset_names = []
steps_per_epoch_list = []
validation_steps_list = []

for combo in dataset_combos:
    training_dataset = None
    val_dataset = None
    train_size = None
    val_size = None
    for idx in combo:
        if training_dataset is None:
            training_dataset = train_datasets[idx]
            val_dataset = val_datasets[idx]
            train_size = train_sizes[idx]
            val_size = val_sizes[idx]
        else:
            training_dataset = training_dataset.concatenate(train_datasets[idx])
            val_dataset = val_dataset.concatenate(val_datasets[idx])
            train_size += train_sizes[idx]
            val_size += val_sizes[idx]
    combined_dataset_names.append("_".join([dataset_names[idx] for idx in combo]))
    combined_training_datasets.append(training_dataset)
    combined_val_datasets.append(val_dataset)
    steps_per_epoch_list.append(train_size // batch_size)
    validation_steps_list.append(val_size // batch_size)

### Brute Force Consolidation

In [12]:
consolidation_combos = []
consolidated_datasets = train_datasets + val_datasets
consolidated_sizes = train_sizes + val_sizes
# Generate combinations ensuring at least one training dataset
for n in range(1, len(consolidated_datasets) + 1):
    for combo in combinations(range(len(consolidated_datasets)), n):
        if any(idx < len(train_datasets) for idx in combo): # Ensure at least one element from train_datasets
            consolidation_combos.append(combo)

print("Consolidation Combos", consolidation_combos)

# Generate consolidated datasets
consolidated_training_datasets = []
consolidated_steps_per_epoch_list = []
consolidated_ids = []
mapping_dict = {}

for id, combo in enumerate(consolidation_combos):
    train_dataset_names = []
    val_dataset_names = []
    consolidated_training_datasets.append(consolidate_to_train([consolidated_datasets[idx] for idx in combo]))
    consolidated_steps_per_epoch_list.append(sum([consolidated_sizes[idx] for idx in combo]) // batch_size)
    if len(combo) == len(consolidated_datasets):
        mapping_dict["all"] = {"train": "all", "val": "all"}
    else:
        for idx in combo:
            normalized_idx = idx % len(dataset_names) # Normalize the index to the dataset_names list to avoid out of bound access
            if idx < len(dataset_names):
                train_dataset_names.append(dataset_names[normalized_idx])
            else:
                val_dataset_names.append(dataset_names[normalized_idx])
        mapping_dict[id] = {"train": train_dataset_names, "val": val_dataset_names}
    consolidated_ids.append(id) # Keeping this to guarantee the order for training loops

# Ensure that the lengths are consistent across the board
assert(len(consolidated_training_datasets) == len(consolidated_steps_per_epoch_list) and len(consolidated_steps_per_epoch_list) == len(consolidated_ids) and len(mapping_dict.keys()) == len(consolidated_ids))

Consolidation Combos [(0,), (1,), (2,), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 2), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5), (0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 3, 4), (0, 3, 5), (0, 4, 5), (1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 3, 4), (1, 3, 5), (1, 4, 5), (2, 3, 4), (2, 3, 5), (2, 4, 5), (0, 1, 2, 3), (0, 1, 2, 4), (0, 1, 2, 5), (0, 1, 3, 4), (0, 1, 3, 5), (0, 1, 4, 5), (0, 2, 3, 4), (0, 2, 3, 5), (0, 2, 4, 5), (0, 3, 4, 5), (1, 2, 3, 4), (1, 2, 3, 5), (1, 2, 4, 5), (1, 3, 4, 5), (2, 3, 4, 5), (0, 1, 2, 3, 4), (0, 1, 2, 3, 5), (0, 1, 2, 4, 5), (0, 1, 3, 4, 5), (0, 2, 3, 4, 5), (1, 2, 3, 4, 5), (0, 1, 2, 3, 4, 5)]


### Generate the test dataset

In [ ]:
if full_test_dir is None:
    test_generators = []
    print("No target test directory provided, merging all tests from provided datasets if available")
    for d in test_dirs:
        if d is not None:
            test_generators.append(create_generators(d, augment=False, shuffle=False, img_height=img_height, img_width=img_width, batch_size=batch_size)[0]) # No augmentation/shuffle for testing
    if len(test_generators) == 0:
        raise ValueError("No tests found in the provided datasets")
    true_labels = np.concatenate([gen.classes for gen in test_generators])
    test_dataset = generators_to_dataset(test_generators, batch_size=batch_size)
    test_steps = sum([gen.samples for gen in test_generators]) // batch_size
    print("Test Dataset Class Counts:")
    for gen in test_generators:
        print("Class indices:", gen.class_indices)
        for class_name, class_index in gen.class_indices.items():
            print(f"{class_name}: {sum(gen.classes == class_index)}")
    print("\n")
    
else:
    test_generator, augmented_test_generator = create_generators(full_test_dir, augment=False, shuffle=False, img_height=img_height, img_width=img_width, batch_size=batch_size) # No augmentation/shuffle for testing
    test_dataset = create_dataset(test_generator, batch_size=batch_size, img_height=img_height, img_width=img_width)
    test_steps = test_generator.samples // batch_size
    true_labels = test_generator.classes
    print("Class indices:", test_generator.class_indices)
    print("\n")
    print("Test Dataset Class Counts:")
    for class_name, class_index in test_generator.class_indices.items():
        print(f"{class_name}: {sum(test_generator.classes == class_index)}")
    print("\n")

true_labels = true_labels[: (len(true_labels) // batch_size) * batch_size] # Ensure that the true labels are divisible by the batch size to avoid size mismatch with predictions.

Found 858 images belonging to 2 classes.
Class indices: {'fire': 0, 'nofire': 1}


Test Dataset Class Counts:
fire: 371
nofire: 487




### Model Preparation

In [14]:
def generate_model(bm, custom=False):
    if custom:
        model = bm
        model.compile(optimizer=optimizer_fn, loss=loss_fn, metrics=metrics_list)
        model.save_weights(os.path.join(checkpoint_path, f"{model.name}_initial.weights.h5"))    
        return model
    
    base_model = bm(
        include_top=False,
        weights='imagenet',
        input_shape=(img_height, img_width, 3)
    )
    base_model.trainable = False

    # Create the model
    inputs = Input(shape=(img_height, img_width, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs, name=bm.__name__)
    model.compile(optimizer=optimizer_fn, loss=loss_fn, metrics=metrics_list)
    model.save_weights(os.path.join(checkpoint_path, f"{model.name}_initial.weights.h5"))    
    return model

### Training and evaluating the models and combinations

In [15]:
training_results = {}
run_number = len([d for d in os.listdir("runs") if os.path.isdir(os.path.join("runs", d)) and d.startswith('run_')]) + 1
run_dir = os.path.join("runs", f"run_{run_number}")
os.makedirs(run_dir, exist_ok=True)

with open(os.path.join(run_dir, 'brute_force_mapping_dict.json'), 'w') as f:
    json.dump(mapping_dict, f, indent=4)

In [ ]:

results_file = os.path.join(run_dir, 'training_results.json')

for base_model, custom_bool in zip(all_models, is_custom_model):
    model = generate_model(base_model, custom=custom_bool) # To display the model summary
    model.summary()
    training_results[model.name] = {}
    plot_model(model, show_shapes=True, show_layer_names=True, to_file=os.path.join("architectures", f"{model.name}_architecture.png"))
    for dataset_id, train_dataset, val_dataset, steps_per_epoch, validation_steps in zip(combined_dataset_names, combined_training_datasets, combined_val_datasets, steps_per_epoch_list, validation_steps_list):
        model.load_weights(os.path.join(checkpoint_path, f"{model.name}_initial.weights.h5"))
        print(f"Training model: {model.name} on dataset: {dataset_id}")
        
        # Record the start time
        start_time = time.time()

        # Initial training of the model
        history = model.fit(
            train_dataset,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            validation_data=val_dataset,
            validation_steps=validation_steps,
            callbacks=callbacks_list
        )

        # Record the end time
        end_time = time.time()
        # Calculate the training time
        training_time = end_time - start_time
        print(f"Training time: {training_time:.2f} seconds")

        model_ds_dir = os.path.join(run_dir, model.name, dataset_id)
        os.makedirs(model_ds_dir, exist_ok=True)
        # Save the model
        model.save(os.path.join(model_ds_dir, f"{model.name}_{dataset_id}.keras"))

        ### Evaluation stage ###
        optimal_threshold = full_eval(model_ds_dir, history, model, dataset_id, test_dataset, true_labels, test_steps)
        
        training_results[model.name][dataset_id] = {
            'history': history.history,
            'training_time': training_time,
            'optimal_threshold': float(optimal_threshold),
            'train_dataset_size': steps_per_epoch * batch_size,
            'val_dataset_size': validation_steps * batch_size,
            "evaluation": model.evaluate(test_dataset, return_dict=True, steps=test_steps)
        }

        # Save the training results to a file after each iteration
        with open(results_file, 'w') as f:
            json.dump(training_results, f, indent=4)
        
        model.compile(optimizer=optimizer_fn, loss=loss_fn, metrics=metrics_list) # Reset the model for the next iteration

Model: "MobileNetV3Small"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MobileNetV3Small (Functional)   │ (None, 7, 7, 576)      │       939,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 576)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 576)            │         2,304 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │       147,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,090,417 (4.16 MB)

 Trainable params: 149,633 (584.50 KB)

 Non-trainable params: 940,784 (3.59 MB)

Training model: MobileNetV3Small on dataset: The Wildfire Dataset
Epoch 1/80
117/117 ━━━━━━━━━━━━━━━━━━━━ 25s 170ms/step - accuracy: 0.6149 - auc: 0.6826 - f1_score: 0.6429 - loss: 0.7686 - precision: 0.7215 - recall: 0.5944 - val_accuracy: 0.5987 - val_auc: 0.6427 - val_f1_score: 0.7453 - val_loss: 0.6687 - val_precision: 0.5987 - val_recall: 1.0000 - learning_rate: 0.0010
Epoch 2/80
117/117 ━━━━━━━━━━━━━━━━━━━━ 9s 75ms/step - accuracy: 0.7151 - auc: 0.7761 - f1_score: 0.7654 - loss: 0.6004 - precision: 0.7753 - recall: 0.7611 - val_accuracy: 0.6037 - val_auc: 0.8335 - val_f1_score: 0.7501 - val_loss: 0.6549 - val_precision: 0.6037 - val_recall: 1.0000 - learning_rate: 0.0010
Epoch 3/80
117/117 ━━━━━━━━━━━━━━━━━━━━ 9s 78ms/step - accuracy: 0.7435 - auc: 0.8065 - f1_score: 0.7914 - loss: 0.5388 - precision: 0.7836 - recall: 0.8048 - val_accuracy: 0.5962 - val_auc: 0.8784 - val_f1_score: 0.7389 - val_loss: 0.6090 - val_precision: 0.5917 - val_recall: 1.0000 - learning_rate: 0.0010
Epoch

In [19]:
print("Brute force loop completed!")
print(f"All models and evaluations are available at: {run_dir}")

Brute force loop completed!
All models and evaluations are available at: runs\run_6
